import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

!unzip -q train.zip -d .

In [3]:
import os
print(len(os.listdir('./data/dogs-vs-cats-redux-kernels-edition/train/')))

25000


In [5]:
import tensorflow as tf
import shutil

In [6]:
for i in os.listdir('./data/dogs-vs-cats-redux-kernels-edition/train/'):
    if 'cat' in i:
        shutil.copyfile('./data/dogs-vs-cats-redux-kernels-edition/train/' + i, './data/dogs-vs-cats-redux-kernels-edition/cat/' + i)
    if 'dog' in i:
        shutil.copyfile('./data/dogs-vs-cats-redux-kernels-edition/train/' + i, './data/dogs-vs-cats-redux-kernels-edition/dog/' + i)

In [14]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    './data/dataset/',
    image_size=(64,64),
    batch_size=64,
    subset='training',
    validation_split=0.2,
    seed=1234
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    './data/dataset/',
    image_size=(64,64),
    batch_size=64,
    subset='validation',
    validation_split=0.2,
    seed=1234
)

print(train_ds)

# 데이터 전처리
# 255를 0~1로 압축 시간을 감소 시켜 보려 했으나 큰 차이가 없네
def 전처리함수(i, 정답):
    i = tf.cast(i/255.0, tf.float32)
    return i, 정답

train_ds = train_ds.map(전처리함수)
val_ds = val_ds.map(전처리함수)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<BatchDataset shapes: ((None, 64, 64, 3), (None,)), types: (tf.float32, tf.int32)>


In [15]:
# import matplotlib.pyplot as plt

for i, a in train_ds.take(1):
    print(i, a)
#     plt.imshow(i[0].numpy().astype('uint8'))
#     plt.show()

tf.Tensor(
[[[[0.08425054 0.02934857 0.02542701]
   [0.0788603  0.02395833 0.02003676]
   [0.09209559 0.04895834 0.03327206]
   ...
   [0.06710516 0.02360026 0.00803654]
   [0.07450981 0.01960784 0.01568628]
   [0.00392923 0.00803654 0.00392923]]

  [[0.1010972  0.0501168  0.02658739]
   [0.119267   0.06828661 0.0447572 ]
   [0.06590265 0.03845167 0.01492226]
   ...
   [0.07071079 0.02720588 0.01164216]
   [0.06977443 0.01487247 0.0109509 ]
   [0.00349839 0.01146408 0.00509153]]

  [[0.09571078 0.04473039 0.01335784]
   [0.09761029 0.0466299  0.01525735]
   [0.07900965 0.066142   0.03922909]
   ...
   [0.07178692 0.02703546 0.01188726]
   [0.0753753  0.02147863 0.01722197]
   [0.00857843 0.01602137 0.00982307]]

  ...

  [[0.3947572  0.32857114 0.24108073]
   [0.46586245 0.3835095  0.26279873]
   [0.5416322  0.4666322  0.34874004]
   ...
   [0.46569967 0.29682714 0.0887389 ]
   [0.37026656 0.20948224 0.0077742 ]
   [0.4407437  0.29987362 0.05557215]]

  [[0.48482114 0.40479666 0.296574

In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2), # 오버피팅 완화 기능 dropout, 20% 제거
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds,epochs=5)

Epoch 1/5
313/313 [==============================] - 171s 545ms/step - loss: 0.6173 - accuracy: 0.6517 - val_loss: 0.5617 - val_accuracy: 0.7124
Epoch 2/5
313/313 [==============================] - 99s 318ms/step - loss: 0.5170 - accuracy: 0.7454 - val_loss: 0.4806 - val_accuracy: 0.7732
Epoch 3/5
313/313 [==============================] - 92s 295ms/step - loss: 0.4679 - accuracy: 0.7783 - val_loss: 0.4776 - val_accuracy: 0.7786
Epoch 4/5
313/313 [==============================] - 91s 290ms/step - loss: 0.4282 - accuracy: 0.8052 - val_loss: 0.4275 - val_accuracy: 0.8098
Epoch 5/5
313/313 [==============================] - 92s 294ms/step - loss: 0.3987 - accuracy: 0.8174 - val_loss: 0.3862 - val_accuracy: 0.8236
